#### Arma clusters y bigramas en base a los utterances de entrada

In [20]:
# SETEO
archivo_csv_importar = 'cruce_intenciones.csv' #nombre del archivo que contiene las oraciones
titulo_columna = 'utterance' #nombre de la columna que contiene las utterances
nombreArchivoExportar = 'cruce_intenciones-bigramas'

#Al trabajar con utterances cuya longitud (en caracteres) es menor a 50, los clusters son mejores
quedarseConUtterancesCortos = False 

# '' default (ej: pagó), 's' stemmization (ej: pag_), 'l' lemmatization (ej: pagar)
forma_tokenizar_clusters = 's' 

#Definir que modelo usar, armar los vectores con tfidf es más rápido
modelo_preentrenado = True # funciona con agglomerativeClustering
tfidf = False # funciona con kmeans a continuación

#Definir modelos para armar los clusters:
kmeans = False  # funciona con tfidf
umaphdbscan = False
agglomerativeClustering = True # funciona con modelo_preentrenado
distance_threshold = 1.5 # se setea si se usa agglomerativeClustering: a cuanto mayor valor -> menor cant de clusters arma

##### SOLO SI SE USA KMEANS:
#Si buscarCantidadDeClusters está en True hay que definir el rango en intervaloBusquedaClusters, sino, se define la cantidad de clusters a mano en cantidadDeClusters.
buscarCantidadDeClusters = False #se realiza la busqueda con silhouette score
intervaloBusquedaClusters = range(2,5)
cantidadDeClusters = 7

#Por si se quieren armar subclusters dentro de los clusters
armarSubClusters = False
armarSubClustersKmeans = False

# 2 (bigramas) o 3 (trigramas)
crear_unigramas = True
crear_bigramas = True
crear_trigramas = False
forma_tokenizar_bigramas = 's' # '' default (ej: pagó), 's' stemmization (ej: pag_), 'l' lemmatization (ej: pagar)

# lista de stopwords para agregar a las que ya existen en la librería
lista_stopwords_agregar = ["hola","Isa","a","eh","días","día","dia","dias","noche","noches","tardes","quiero","quería","queria","quisiera"
                   ,"necesito","necesitaría","necesitaria","yo","llamo","llame","llamé","llamaba","quisiera","ok","mirá","mira","aca","acá"
                   ,"decirte","favor","preguntar"]

#sinonimos que se usan si se vectoriza con TFIDF
sinonimos = {'agente': 'representante', 'persona': 'representante', 'operador': 'representante', 'asesor': 'representante'
             , 'empleado': 'representante', 'humano': 'representante', 'alguien': 'representante', 'atención': 'representante'
             , 'máquina': 'representante', 'gerente': 'representante', 'asistente': 'representante', 'sector': 'representante'
             , 'abonar': 'pagar', 'saldar': 'pagar', 'pago': 'pagar'
             , 'monto': 'saldo', 'deuda': 'saldo', 'mes': 'saldo', 'debo': 'saldo', 'importe': 'saldo', 'costo': 'saldo'
             , 'adeudo': 'saldo', 'cuánto': 'saldo'
             , 'telefono': 'linea', 'celular': 'linea', 'personal': 'linea', 'lineas': 'linea', 'telefinica': 'linea'
             , 'chip': 'linea', 'sms': 'linea'
             , 'wi-fi': 'internet', 'whatsapp': 'internet', 'adsl': 'internet', 'router': 'internet', 'modem': 'internet'
             , 'wifi': 'internet', 'wi fi': 'internet'
             , 'fibertel': 'internet', 'gigas': 'internet', 'giga': 'internet', 'paquete': 'internet'
             , 'television': 'cable', 'televisor': 'cable', 'canales': 'cable', 'canal': 'cable', 'flow': 'cable'
             , 'disney': 'cable', 'hbo': 'cable', 'cablevision': 'cable', 'futbol': 'cable', 'hd': 'cable', '4k': 'cable', 'pack': 'cable'
             , 'canal': 'cable', 'canales': 'cable'
             , 'boleta': 'factura', 'boletas': 'factura', 'bolita': 'factura', 'papeleta': 'factura', 'abono': 'factura'
             , 'plan': 'factura', 'facturas': 'factura', 'facturacion': 'factura'
             , 'consulta': 'consultar', 'conocer': 'consultar', 'saber': 'consultar', 'averiguar': 'consultar', 'ver': 'consultar'
             , 'preguntar': 'consultar', 'pregunta': 'consultar'
             , 'comunicar': 'hablar', 'pasar': 'hablar', 'atender': 'hablar', 'atienda': 'hablar', 'dame': 'hablar'
             , 'pásame': 'hablar', 'pasame': 'hablar'
             , 'quería': 'quiero', 'quisiera': 'quiero', 'necesito': 'quiero', 'necesitaría': 'quiero', 'gustaria': 'quiero'
             , 'error': 'problema', 'inconveniente': 'problema', 'suspendido': 'problema', 'restauracion': 'problema'
             , 'restaurar': 'problema', 'resolver': 'problema', 'recorte': 'problema', 'recuperar': 'problema', 'reclamo': 'problema'
             , 'vencido': 'vencida', 'vencimiento': 'vencida', 'vence': 'vencida'
             , 'habilitar': 'activar', 'activado': 'activar', 'habilitado': 'activar'
             , 'debaja': 'baja', 'debajo': 'baja', 'quitar': 'baja', 'eliminar': 'baja', 'borrar': 'baja', 'borrarme': 'baja'
             , 'descuento': 'promocion', 'rebaja': 'promocion', 'bonificacion': 'promocion', 'beneficio': 'promocion'
             , 'promo': 'promocion', 'barato': 'promocion'
             , 'bastante': 'aumento', 'caro': 'aumento', 'mas': 'aumento', 'tanto': 'aumento', 'interes': 'aumento'
             , 'mucho': 'aumento', 'aumento': 'aumento', 'incremento': 'aumento', 'incremento': 'aumento'
             , 'llego': 'aumento', 'cobrando': 'aumento', 'cobro': 'aumento', 'pagando': 'aumento', 'vino': 'aumento'
             , 'hola': 'saludo', 'tardes': 'saludo', 'dias': 'saludo', 'noches': 'saludo'
             , 'asunto': 'tema'
             , 'sin': 'no'
             , 'anda': 'funciona', 'prende': 'funciona', 'enciende': 'funciona', 'apaga': 'funciona', 'apagado': 'funciona'  
             , 'pixel': 'imagen', 'pixela': 'imagen', 'pantalla': 'imagen'
             , 'codificador': 'decodificador', 'deco': 'decodificador', 'decodificadores': 'decodificador', 'codificadores': 'decodificador', 'aparato': 'decodificador' }

In [21]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime, timedelta
# import random 
from random import sample
import os, ibm_db, ibm_db_dbi as dbi
import operator
from collections import Counter
import string
from gensim.models.phrases import Phrases, Phraser
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

import torch
from sklearn.cluster import AgglomerativeClustering


In [22]:
from project_lib import Project
projectEtl = Project(project_id="a4f6b980-d41e-4046-88de-e420dbf240ff",project_access_token= "p-5ccb6ac487a9511c00551cdd67a943556f8b77ff") #ETL
projectCCoC = Project(project_id='8568290c-dd4e-4248-8e07-a29c6a22794a', project_access_token='p-4f9aa7521c92c00bded127a17a3044db94003d99') #centro competencias
pc1 = projectEtl.project_context
pc2 = projectCCoC.project_context
Db2Certificate_metadata = projectEtl.get_connection(name="CCC_Db2")

In [23]:
!pip3 install nltk

In [24]:
pip install transformers==4.15.0

  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib/tokenizers
      creating build/lib/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib/tokenizers/models
      creating build/lib/tokenizers/decoders
      copying py_src/tokenizers/decoders/__init__.py -> build/lib/tokenizers/decoders
      creating build/lib/tokenizers/normalizers
      copying py_src/tokenizers/normalizers/__init__.py -> build/lib/t

In [25]:
pip install -U sentence-transformers #instalar librería para vectorizar textos con modelos preentrenados

Note: you may need to restart the kernel to use updated packages.


In [26]:
# pip install umap-learn
# pip install hdbscan
# import umap
# import hdbscan

In [27]:
import random
from functools import partial

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from tqdm.notebook import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

pd.set_option("display.max_rows", 600)
pd.set_option("display.max_columns", 500)
pd.set_option("max_colwidth", 400)

In [28]:
#pip install spacy-transformers

In [29]:
from sentence_transformers import SentenceTransformer, util #se importan
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [30]:
import nltk

In [31]:
# !pip install spacy==3.1.0

# #SÓLO CORRER ESTA CELDA LA PRIMERA VEZ (UNA VEZ QUE SE SUBIÓ EL "es_core_news_sm-3.1.0.tar.gz"). LUEGO NO HACE FALTA CORRER ESTA CELDA
# with open("es_core_news_sm-3.1.0.tar.gz","wb") as f:
#     f.write(projectEtl.get_file("es_core_news_sm-3.1.0.tar.gz").read())
# !pip install es_core_news_sm-3.1.0.tar.gz


In [32]:
# import spacy
# from spacy.lang.es.stop_words import STOP_WORDS
# nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])

In [33]:
# limpiar puntuaciones y stopwords
punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords = stopwords.flatten().tolist()
#lista_stopwords = ["hola","eh", "pago", "pagar","quiero","factura","linea","queria", 
#          "necesito","quisiera","numero","abono","boleta","dia","tarjeta","gracias","importe","mes","monto","deuda","a"]

stopwords.extend(lista_stopwords_agregar) #se agrega la lista de stopwords seteada anteriormente

#stopwords para remover de la lista
stopwords.remove("cuanto")
stopwords.remove("tanto")
stopwords.remove("poner")
stopwords.remove("mismo")
#stopwords.remove("no")

In [34]:
from nltk.stem import * #SnowballStemmer
stemmer = SnowballStemmer("spanish")

In [35]:
def normalize(s): #Elimina tildes y puntuaciones
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    s = s.translate(str.maketrans('', '', punctuation)).lower()#.split() #se sacan los signos de puntuación
    return s

def simple_tokenizer(doc,stemm=False, lem=False): #elimina dígitos, elimina stopwords, utiliza stemming o lemmatizacion. Devuelve una lista de palabras
    tokenized_doc = ["" if t.isdigit() else t for t in doc.split()]  # se remueven los números
    tokenized_doc = [
        token.lower() for token in tokenized_doc if token.lower() not in stopwords and token != ""
    ] #se sacan las stopwords y ademas los tokens que quedan como "" ya que son números
    
    if stemm:
        tokenized_doc = [stemmer.stem(str(token)) for token in tokenized_doc] #sólo el stemming de los tokens
        
    if lem: #lemmatization
        tokenized_doc = " ".join(tokenized_doc)
        tokenized_doc = [x.lemma_ for x in nlp(tokenized_doc)]
        
    return tokenized_doc

def simple_tokenizer_cluster(doc,stemm=False, lem=False): #mismo que arriba pero devuelve oracion en vez de lista de palabras
    text_limpio = simple_tokenizer(doc,stemm, lem)
    text_limpio = " ".join(text_limpio)
    return text_limpio

def ngramas(df,gramas,nombre_columna_bigrama,nombre_columna_repeticion): #crea los bigramas o unigramas y el porcentaje que representan respecto al total
    df[f'{gramas}_gramas'] = df['tokens'].apply(lambda row: list(nltk.ngrams(row, gramas))) #se crean los ngramas para cada fila
    lista = [] #se crea una lista vacía para ir guardando los bigramas
    def desarmar(lista_):
        for conjuntos in set(lista_):
            lista.append(conjuntos)

    df[f'{gramas}_gramas'].apply(lambda x: desarmar(x)) #con lo siguiente se van pasando los bigramas a la función desarmar() y se guardan en una lista
    frequence = nltk.FreqDist(lista) #se calcula la frecuencia de aparición de los bigramas en la lista creada anteriormente
    sort = sorted(frequence.items(), key=operator.itemgetter(1), reverse=True) #se ordenan de mayor frecuencia a menor
    df[f'{gramas}_gramas_str'] = df[f'{gramas}_gramas'].apply(lambda x: [str(i) for i in x]) #se convierte la columna de gramas a string
    bigramas = pd.DataFrame([[str(i[0]), i[1]] for i in sort]) # la variable sort contiene el bigrama y la frecuencia de aparición, en esta línea se crea un df
    bigramas.rename(columns={0:nombre_columna_bigrama,1:"cantidad"},inplace=True)
    #que contiene el bigrama y su aparición
    df[nombre_columna_bigrama] = df[f'{gramas}_gramas_str'].apply(lambda x: next((i for i in bigramas[nombre_columna_bigrama].to_list() if i in x), None))
    #df["repeticion"] = df['bigrama'].apply(lambda x: bigramas[bigramas["bigrama"]==x]["cantidad"].iloc[0] if x != None else 0)
    repeticiones = df[nombre_columna_bigrama].value_counts()
    df[nombre_columna_repeticion] = df[nombre_columna_bigrama].map(repeticiones)
    #df["repeticion"].fillna(0)
    
    return sort

def buscarYDevolverIntencionInicialConfianza(context):  #cuanto se baja data de DB2, para extraer la confianza
    accion = re.findall("'confidence':0.[0-9]+", context)
    if len(accion)==1:
        return re.findall("0.[0-9]+", accion[0])[0]
    else: 
        accion = re.findall("'confidence':[0-9]+", context)
        if len(accion)==2:
            return re.findall("[0-9]+", accion[0])[0]
        else: return ""

def buscarYDevolverIntencionInicial(context): #cuanto se baja data de DB2, para extraer la intencion
    accion = re.findall("'intent':'(\w+)'", context)
    if len(accion)==1:
        return re.findall("(\w+)", accion[0])[0]
    else: return ""

#### Levantar CSV

In [36]:
#se levanta el csv:
utterances = projectCCoC.get_file(archivo_csv_importar) 
utterances = pd.read_csv(utterances)

In [37]:
columnas_originales_del_csv = utterances.columns

In [38]:
utterances[titulo_columna] = utterances[titulo_columna].apply(lambda x: normalize(x)) #normalizo los textos

In [39]:
utterances["longitud"] = utterances[titulo_columna].apply(lambda x: len(x)) #longitud de los textos

In [40]:
if quedarseConUtterancesCortos:  
    utterances_largos=utterances[utterances["longitud"]>50].copy() #utterances cuya longitud es mayor a 50
    utterances=utterances[utterances["longitud"]<=50].copy() #utterances cuya longitud es menor a 50. Con estos se trabaja de acá en adelante.

In [41]:
#se eliminan espacios en blanco que a veces quedan en las oraciones
def eliminarEspacioBlanco(texto):
    palabras = texto.split()
    texto_limpio = ' '.join(palabra.strip() for palabra in palabras)
    return texto_limpio
utterances[titulo_columna] = utterances[titulo_columna].apply(lambda x: eliminarEspacioBlanco(x))

In [42]:
#se eliminan textos iguales (se deja uno solo)
utterances.drop_duplicates(subset=[titulo_columna],inplace=True)
utterances.reset_index(inplace=True)

In [43]:
#Por si se quiere aplicar stemming o lemmatizacion a las oraciones
if forma_tokenizar_clusters == 's':
    utterances["tokens_cluster"] = utterances.apply(lambda x: simple_tokenizer_cluster(x[titulo_columna],stemm=True), axis = 1) #se tokenizan los textos, opcion stem
elif forma_tokenizar_clusters == 'l':
    utterances["tokens_cluster"] = utterances.apply(lambda x: simple_tokenizer_cluster(x[titulo_columna],lem=True), axis = 1) #se tokenizan los textos, opcion lem
else:
    utterances["tokens_cluster"] = utterances.apply(lambda x: simple_tokenizer_cluster(x[titulo_columna]), axis = 1) #se tokenizan los textos, normal

In [44]:
if "index" in utterances.columns:
    utterances.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances.columns:
    utterances.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances.columns:
    utterances.drop(columns=['level_0'],inplace=True)

In [45]:
len(utterances)

302

In [46]:
utterances

,utterance,correct_intent,top_intent,top_confidence,longitud,tokens_cluster
0,buscando un equipo nuevo que es lo que me pueden ofrecer,compraEquipo,(skill),NaN,56,busc equip ofrec
1,tenes algun celular en promo en stock,compraEquipo,(skill),NaN,37,ten algun celul prom stock
2,se me rompio el celular y necesito cambiarlo quiero reponerlo,compraEquipo,(skill),NaN,61,rompi celul cambi repon
3,saber si tienen el iphone,compraEquipo,(skill),NaN,25,iphon
4,saber si puedo sacar un equipo nuevo,compraEquipo,(skill),NaN,36,sac equip
5,saber si con esta linea puedo comprar otro celular,compraEquipo,(skill),NaN,50,line compr celul
6,saber si alguien tiene stock de un telefono,compraEquipo,(skill),NaN,43,algui stock telefon
7,saber para comprar un telefono a traves de factura,compraEquipo,(skill),NaN,50,compr telefon trav factur
8,saber el precio de un aparato nuevo,compraEquipo,(skill),NaN,35,preci aparat
9,quisiera comprar un equipo en cuotas,compraEquipo,(skill),NaN,36,compr equip cuot


### Armar clusters

#### Vectorizar con sentence transformers:


In [47]:
#Se vectorizan los textos con un modelo preentrenado de sentence transformers
if modelo_preentrenado:
    X = model.encode(utterances[titulo_columna].to_list()) 

#### Vectorizar con TF-IDF:

In [48]:
def synonym_tokenizer(text):
    words = text.split()
    tokens = [sinonimos.get(word, word) for word in words]
    return tokens

In [49]:
#se vectorizan los textos con TFIDF
if tfidf:

    vectorizer = TfidfVectorizer(sublinear_tf= True, min_df=10, norm='l2', ngram_range=(1, 2), encoding='utf-8'
                                 , decode_error='ignore', strip_accents='ascii', lowercase=True
                                 , tokenizer=synonym_tokenizer #función de sinónimos
                                 , stop_words=None) #stop_words='spanish')
    X = vectorizer.fit_transform(utterances["tokens_cluster"])

#### Continúa clusterización

In [50]:
# #Si se quiere realizar umap y luego hdbscan, se realiza la busqueda de hiperparametros
# if umaphdbscan:
#     hspace = {
#             "n_neighbors": hp.choice('n_neighbors', range(3,16)),
#             "n_components": hp.choice('n_components', range(100,101)), #dejar fijo este valor, es decir que la reducción de dimensionalidad sea de 99. Ver como modificarlo para que no pruebe
#             "min_cluster_size": hp.choice('min_cluster_size', range(3,16)), #acá se setea dependiendo del tamaño del dataset, ya que determina el valor mínimo de datos que un cluster debe tener
#             "random_state": 42
#         }

#     label_lower = 15
#     label_upper = 25
#     max_evals = 100

#     best_params_use, best_clusters_use, trials_use = bayesian_search(X, 
#                                                                      space=hspace, 
#                                                                      label_lower=label_lower, 
#                                                                      label_upper=label_upper, 
#                                                                      max_evals=max_evals)
    
#     clusters_ = generate_clusters__(X,n_neighbors=best_params_use["n_neighbors"],n_components=best_params_use["n_components"],min_cluster_size=best_params_use["min_cluster_size"],random_state=42)
    
#     clusters_ = generate_clusters__(X,n_neighbors=best_params_use["n_neighbors"],n_components=best_params_use["n_components"],min_cluster_size=best_params_use["min_cluster_size"],random_state=42)

In [51]:
##original=utterances.copy()

# utterances = original.head(10000).copy()

In [52]:
# longitud = utterances.longitud.to_list()

In [53]:
# longitud_normalizada = [i / max(longitud) for i in longitud]
# longitud_normalizada = np.array(longitud_normalizada)

# longitud_normalizada = np.expand_dims(longitud_normalizada, axis=-1)

In [54]:
#clustering con agglomerativeClustering
if agglomerativeClustering:
    
    print ("Inicio: ", datetime.now() - timedelta(hours = 3) )

    corpus_embeddings = model.encode(utterances[titulo_columna].to_list())
    # Normalize the embeddings to unit length
    corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
    
    #longitud_normalizada = np.expand_dims(longitud_normalizada, axis=-1)
    
    #corpus_embeddings = np.concatenate((corpus_embeddings, longitud_normalizada), axis=1)
    
    #pca = PCA(n_components=500)

    #corpus_embeddings = pca.fit_transform(corpus_embeddings)

    # Perform clustering
    clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_

    utterances["cluster"] = cluster_assignment
    
    print ("Fin: ", datetime.now() - timedelta(hours = 3) )


Inicio:  2023-10-09 12:13:35.924942
Fin:  2023-10-09 12:13:36.108024


In [55]:
#utterances["cluster"].nunique()

In [56]:
# utterances[utterances["cluster"]==3].FCD_COG_INPUT_TEXT.to_list()

In [57]:
from sentence_transformers import util

In [58]:
#subclusters
utterances_ordenado = pd.DataFrame()
textos_representativos = pd.DataFrame(columns=["oracion","cluster","subcluster","promedio","cantidad_oraciones"]) #para ir guardando la oración más representativa por subcluster
if armarSubClusters:
    for cluster in range(0,len(utterances["cluster"].value_counts())): #se recorren los clusters
        
        df_cluster = utterances[utterances["cluster"]==cluster].copy() #se crea un df con sólo el cluster 
        
        df_cluster.reset_index(inplace=True) #se resetea el índice
        
        corpus_embeddings = model.encode(df_cluster[titulo_columna].to_list()) #embeddings de las oraciones del cluster

        corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True) #se normalizan los embeddings

        
        clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2.5) #se define el modelo
        clustering_model.fit(corpus_embeddings) #se arman los clusters
        cluster_assignment = clustering_model.labels_ #se extraen los labels
        utterances.loc[utterances['cluster'] == cluster, 'subcluster'] = cluster_assignment #se asignan los labels en el df general, en la columna "subcluster"
        
        df_cluster.loc[df_cluster['cluster'] == cluster, 'subcluster'] = cluster_assignment #se asignan los labels en el df general, en la columna "subcluster"
        
        for subcluster in range(0,len(df_cluster["subcluster"].value_counts())): #se recorren los subcluster creados 
            
            df_subcluster=df_cluster[df_cluster["subcluster"]==subcluster].copy() #se cera un df con sólo el subcluster
            
            df_subcluster.reset_index(inplace=True) #se resetea el índice
            
            corpus_embeddings = model.encode(df_subcluster[titulo_columna].to_list()) #embeddings de las oraciones de los subclusters
        
            #avg_embedding = torch.mean(torch.tensor(corpus_embeddings), dim=0) #se calcula el vector promedio entre todos los embeddings creados 
            
            #most_representative_sentence = df_subcluster.iloc[torch.argmax(util.cos_sim(avg_embedding, corpus_embeddings)).item()][titulo_columna] #la oracion más representativa de cada subcluster (la mas similar al vector promedio)
            
            # calcular la similitud entre todas las oraciones 
            similarity_matrix = util.cos_sim(corpus_embeddings, corpus_embeddings)

            promedios = [i.mean().item() for i in similarity_matrix] #promedio de similitud por oracion. Es decir, en promedio cuanto se parece la oracion al resto del subcluster
            
            most_representative_sentence = df_subcluster.iloc[promedios.index(max(promedios))][titulo_columna] #oracion con el promedio más alto. La tomo como la más representativa del subcluster

            promedio_general = sum(promedios) / len(promedios) #promedio de promedios. Si el promedio general es bajo significa que las oraciones no guardan mucha relación entre si por lo que se los podría considerar outliers
                                                                    
            
            new_row = {'oracion': most_representative_sentence, 'cluster': cluster, 'subcluster': subcluster,'promedio':promedio_general,'cantidad_oraciones':len(df_subcluster)}
            #textos_representativos = textos_representativos.append(new_row, ignore_index=True) #se asignan al df
            textos_representativos = pd.concat([textos_representativos, pd.DataFrame([new_row])], ignore_index=True)
            
            threshold = 0.75
            indices = []
            x = 0 
            i=0
            while i<len(similarity_matrix):
                if x not in indices: #se fija si el indice de la oración ya está guardado en la lista
                    indices.append(x) #si NO está guardado, lo guarda
                for z in reversed(similarity_matrix.argsort()[x]): #recorro la matriz de similitud para esa oración (desde el más similar al menos)
                    if z.item() in indices: #si el índice ya existe en la lista, sigo recorriendo la matriz
                        continue
                    else: 
                        indices.append(z.item()) #una vez que encuentra un índice que no está, lo guarda en la lista 
                        x = z.item() #asigna a "x" ese índice
                        break #sale del bucle for
                i += 1 #suma 1 a "i"
                
                
            df_subcluster__ = df_subcluster.iloc[indices]

            # resetear índices
            df_subcluster__ = df_subcluster__.reset_index(drop=True)

            #utterances_ordenado=utterances_ordenado.append(df_subcluster__)
            
            utterances_ordenado = pd.concat([utterances_ordenado, df_subcluster__], ignore_index=True)

else:
    
    utterances_ordenado = utterances.copy()
        

### Podría quedarme con los clusters cuyo promedio supera algún umbral y con los otros armar subclusters

In [59]:
textos_representativos.sort_values(["promedio"],ascending=False)


,oracion,cluster,subcluster,promedio,cantidad_oraciones


In [60]:
textos_representativos[textos_representativos["promedio"]>=0.55]

,oracion,cluster,subcluster,promedio,cantidad_oraciones


In [61]:
pip install xlsxwriter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 19.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [62]:
# generarExcel([utterances_ordenado,textos_representativos],["textos y clusters","texto representativo"],"clusters_IR_noEntendimiento")

In [63]:
if "index" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=['level_0'],inplace=True)

In [64]:
# utterances["outliers"] = 0
# # calcular la similitud entre todas las oraciones del cluster
# similarity_matrix = util.cos_sim(X, X)

# # establecer umbral de similitud mínima
# threshold = 0.35

# # descartar oraciones con menor similitud
# for i in range(len(utterances)):
#     sim = similarity_matrix[i]
#     sim[i] = 0  # eliminar similitud consigo mismo
#     if sim.mean() < threshold:
#         utterances.at[i, "outliers"] = 1  # marcar oración como outlier

In [65]:
#si se utiliza kmeans, para poder definir un numero de cluster se utiliza silhouette score
if kmeans:
    if buscarCantidadDeClusters:  
        from sklearn.metrics import silhouette_score
        import matplotlib.pyplot as plt

        # Definimos una lista de valores de k que queremos evaluar
        k_values = intervaloBusquedaClusters #range(desde,hasta)

        # Calculamos la puntuación de silueta para cada valor de k
        silhouette_scores = []
        for k in k_values:
            kmeans = KMeans(n_clusters=k)
            labels = kmeans.fit_predict(X)
            score = silhouette_score(X, labels)
            silhouette_scores.append(score)

        # Graficamos la puntuación de silueta en función de k
        plt.plot(k_values, silhouette_scores)
        plt.xlabel('Número de clusters (k)')
        plt.ylabel('Puntuación de silueta')
        plt.show()

        # Seleccionamos el valor de k que maximiza la puntuación de silueta
        best_k = k_values[np.argmax(silhouette_scores)]
        print('El mejor valor de k es:', best_k)

In [66]:
#SE ARMAN LOS CLUSTERS con kmeans
if kmeans: 
    if not buscarCantidadDeClusters:
        best_k = cantidadDeClusters


    # initialize kmeans
    kmeans = KMeans(n_clusters=best_k, random_state=42)
    # fit the model
    kmeans.fit(X)
    # store cluster labels in a variable
    clusters = kmeans.labels_

    # initialize PCA with 2 components
    # pca = PCA(n_components=2, random_state=42)
    # # pass our X to the pca and store the reduced vectors into pca_vecs
    # pca_vecs = pca.fit_transform(X) #X.toarray() PARA TFID
    # # save our two dimensions into x0 and x1
    # x0 = pca_vecs[:, 0]
    # x1 = pca_vecs[:, 1]

    # assign clusters and pca vectors to our dataframe 
    utterances['cluster'] = clusters
    
    utterances_ordenado = utterances.copy()

In [67]:
#subclusters
#cantidad de subcluster por cluster:
if armarSubClustersKmeans:
    cantidad_sub = 3
    for cluster in range(best_k):
        cluster_df = utterances[utterances['cluster'] == cluster]
        embeddings = vectorizer.fit_transform(cluster_df["tokens_cluster"])
        kmeans_subclusters = KMeans(n_clusters=cantidad_sub, random_state=42)
        kmeans_subclusters.fit(embeddings)
        subclusters = kmeans_subclusters.labels_
        utterances.loc[utterances['cluster'] == cluster, 'subcluster'] = subclusters

In [68]:
if kmeans:   
    sklearn_pca = PCA(n_components = 2)
    Y_sklearn = sklearn_pca.fit_transform(X.toarray())
    kmeans = KMeans(n_clusters=best_k, max_iter=600, algorithm = 'auto')
    fitted = kmeans.fit(Y_sklearn)
    prediction = kmeans.predict(Y_sklearn)
    plt.figure(figsize=(30, 15))
    plt.scatter(Y_sklearn[:, 0], Y_sklearn[:, 1], c=prediction, s=40, cmap='viridis', linewidths=5)
    centers = fitted.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1],c='black', s=200, alpha=0.6)
    for i, center in enumerate(centers):
        plt.text(center[0], center[1], f"{i}", fontsize=8, ha='center', va='center', bbox=dict(fc='w', boxstyle='round,pad=1'))

In [69]:
#silhouette score pro cluster
if kmeans:
    from sklearn.metrics import silhouette_score
    from sklearn.metrics import silhouette_samples

    cluster_silhouette_avgs = {}
    # Calcular la puntuación de silueta para cada punto de datos
    silhouette_vals = silhouette_samples(X, clusters)
    for i in range(best_k):
        # Obtener la puntuación de silueta media para el cluster i-ésimo
        cluster_silhouette_vals = silhouette_vals[clusters == i]
        cluster_silhouette_avg = np.mean(cluster_silhouette_vals)
        # Agregar la puntuación de silueta media al diccionario con la clave del número de cluster
        cluster_silhouette_avgs[i] = cluster_silhouette_avg

    # Ordenar el diccionario por valor en orden descendente y mostrar los resultados
    sorted_cluster_silhouette_avgs = sorted(cluster_silhouette_avgs.items(), key=lambda x: x[1], reverse=True)
    for cluster, silhouette_avg in sorted_cluster_silhouette_avgs:
        print("La puntuación de silueta media para el cluster", cluster, "es:", silhouette_avg)

#1 significa que los puntos están muy cerca de su propio cluster y lejos de otros clusteres, mientras que -1 indica que los puntos están cerca de los clusteres vecinos.
#0 indica que en ese cluster hay puntos que pueden pertenecer a otros clusters

### Bigramas/unigramas por clusters

In [70]:
#Se tokenizan las oraciones
if forma_tokenizar_bigramas == 's':
    utterances_ordenado["tokens"] = utterances_ordenado.apply(lambda x: simple_tokenizer(x[titulo_columna],stemm=True), axis = 1) #se tokenizan los textos, opcion stem
elif forma_tokenizar_bigramas == 'l':
    utterances_ordenado["tokens"] = utterances_ordenado.apply(lambda x: simple_tokenizer(x[titulo_columna],lem=True), axis = 1) #se tokenizan los textos, opcion lem
else:
    utterances_ordenado["tokens"] = utterances_ordenado.apply(lambda x: simple_tokenizer(x[titulo_columna]), axis = 1) #se tokenizan los textos, normal

In [71]:
utterances_ordenado.columns

Index(['utterance', 'correct_intent', 'top_intent', 'top_confidence',
       'longitud', 'tokens_cluster', 'cluster', 'tokens'],
      dtype='object')

In [72]:
if "index" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=['level_0'],inplace=True)

In [73]:
titulo_columna_cluster = "cluster"
utterances_ordenado.reset_index(inplace=True)

In [74]:
utterances_ordenado.columns

Index(['index', 'utterance', 'correct_intent', 'top_intent', 'top_confidence',
       'longitud', 'tokens_cluster', 'cluster', 'tokens'],
      dtype='object')

In [75]:
if "index" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances_ordenado.columns:
    utterances_ordenado.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances_ordenado.columns:
    uttutterances_ordenadoerances.drop(columns=['level_0'],inplace=True)

In [76]:
# si se hacen los unigramas por cluster o no
if crear_unigramas:
    n_gramas = 1
    if titulo_columna_cluster == '':
        a = ngramas(utterances, n_gramas,"unigramaCluster")
    else:
        df_final = pd.DataFrame()
        utterances_ = utterances_ordenado.groupby(titulo_columna_cluster)
        for i in utterances_[titulo_columna_cluster].value_counts().index:
            df_cluster = utterances_.get_group(i[0])
            if not all(not sublist for sublist in df_cluster.tokens.to_list()): #a veces pasa que un cluster no tiene valores en "tokens" entonces rompe
                #una lista vacía en Python se evalua como False. Por lo que al hacer "not []" da true. Si todas las listas de tokens del cluster están vacías, el all() devuelve True.
                a = ngramas(df_cluster, n_gramas,"unigramaCluster","repeticionUnigramasClusters")
                df_final = pd.concat([df_final, df_cluster], ignore_index=True)
        df_final = df_final.reset_index()
        utterances_ = df_final.copy()

In [77]:
utterances_ordenado[utterances_ordenado["cluster"]==25]

,utterance,correct_intent,top_intent,top_confidence,longitud,tokens_cluster,cluster,tokens


In [78]:
utterances_[utterances_["cluster"]==25]

,index,utterance,correct_intent,top_intent,top_confidence,longitud,tokens_cluster,cluster,tokens,1_gramas,1_gramas_str,unigramaCluster,repeticionUnigramasClusters


In [79]:
conteo_cluster=pd.DataFrame(utterances_["cluster"].value_counts()).reset_index()
utterances_["porcentajeUnigramaRespectoCluster"] = utterances_.apply(lambda x: (x["repeticionUnigramasClusters"] / conteo_cluster[conteo_cluster["index"]==x["cluster"]]["cluster"].iloc[0])*100,axis=1)

In [80]:
# si se hacen los bigramas por cluster o no
if crear_bigramas:
    n_gramas = 2
    if titulo_columna_cluster == '':
        a = ngramas(utterances, n_gramas,"bigramasClusters")
    else:
        df_final = pd.DataFrame()
        utterances_ = utterances_.groupby(titulo_columna_cluster)
        for i in utterances_[titulo_columna_cluster].value_counts().index:
            df_cluster = utterances_.get_group(i[0])
            if not all(len(sublist)<2 for sublist in df_cluster.tokens.to_list()):
                a = ngramas(df_cluster, n_gramas,"bigramasClusters","repeticionBigramasClusters")
                df_final = pd.concat([df_final, df_cluster], ignore_index=True)
        df_final = df_final.reset_index()
        utterances_ = df_final.copy()

In [81]:
conteo_cluster=pd.DataFrame(utterances["cluster"].value_counts()).reset_index()
utterances_["porcentajeBigramaRespectoCluster"] = utterances_.apply(lambda x: (x["repeticionBigramasClusters"] / conteo_cluster[conteo_cluster["index"]==x["cluster"]]["cluster"].iloc[0])*100,axis=1)

In [82]:
if "index" in utterances_.columns:
    utterances_.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances_.columns:
    utterances_.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances_.columns:
    utterances_.drop(columns=['level_0'],inplace=True)

In [83]:
# si se hacen los trigramas por cluster o no
if crear_trigramas:
    n_gramas = 3
    if titulo_columna_cluster == '':
        a = ngramas(utterances, n_gramas,"trigramasClusters")
    else:
        df_final = pd.DataFrame()
        utterances = utterances.groupby(titulo_columna_cluster)
        for i in utterances[titulo_columna_cluster].value_counts().index:
            df_cluster = utterances.get_group(i[0])
            a = ngramas(df_cluster, n_gramas,"trigramasClusters","repeticionTrigramasClusters")
            df_final = pd.concat([df_final, df_cluster], ignore_index=True)
        df_final = df_final.reset_index()
        utterances = df_final.copy()

In [84]:
utterances

,utterance,correct_intent,top_intent,top_confidence,longitud,tokens_cluster,cluster
0,buscando un equipo nuevo que es lo que me pueden ofrecer,compraEquipo,(skill),NaN,56,busc equip ofrec,5
1,tenes algun celular en promo en stock,compraEquipo,(skill),NaN,37,ten algun celul prom stock,9
2,se me rompio el celular y necesito cambiarlo quiero reponerlo,compraEquipo,(skill),NaN,61,rompi celul cambi repon,0
3,saber si tienen el iphone,compraEquipo,(skill),NaN,25,iphon,13
4,saber si puedo sacar un equipo nuevo,compraEquipo,(skill),NaN,36,sac equip,5
5,saber si con esta linea puedo comprar otro celular,compraEquipo,(skill),NaN,50,line compr celul,9
6,saber si alguien tiene stock de un telefono,compraEquipo,(skill),NaN,43,algui stock telefon,4
7,saber para comprar un telefono a traves de factura,compraEquipo,(skill),NaN,50,compr telefon trav factur,16
8,saber el precio de un aparato nuevo,compraEquipo,(skill),NaN,35,preci aparat,15
9,quisiera comprar un equipo en cuotas,compraEquipo,(skill),NaN,36,compr equip cuot,17


In [85]:
tabla_bigramas_clusters=utterances_[["cluster","bigramasClusters","repeticionBigramasClusters","porcentajeBigramaRespectoCluster"]].drop_duplicates().sort_values(["cluster","porcentajeBigramaRespectoCluster"],ascending=False)
tabla_unigramas_clusters=utterances_[["cluster","unigramaCluster","repeticionUnigramasClusters","porcentajeUnigramaRespectoCluster"]].drop_duplicates().sort_values(["cluster","porcentajeUnigramaRespectoCluster"],ascending=[False,False])

In [86]:
tabla_unigramas=tabla_unigramas_clusters.groupby(["cluster"]).agg({'porcentajeUnigramaRespectoCluster':'max',"unigramaCluster":"first"})[["unigramaCluster","porcentajeUnigramaRespectoCluster"]].reset_index()

In [87]:
tabla_unigramas.groupby(["unigramaCluster"])["cluster"].unique() #cluster en los que se repiten los unigramas

unigramaCluster
('acab',)                     [18]
('aparat',)                   [15]
('cambi',)                    [21]
('celul',)           [0, 2, 6, 12]
('chip',)                     [10]
('compr',)      [8, 9, 13, 16, 17]
('equip',)           [1, 5, 7, 22]
('factur',)               [14, 19]
('limit',)                    [23]
('rob',)                      [20]
('sac',)                       [3]
('telefon',)               [4, 11]
Name: cluster, dtype: object

In [88]:
tabla_unigramas #en qué porcentaje de utterances existe el unigrama por cluster

,cluster,unigramaCluster,porcentajeUnigramaRespectoCluster
0,0,"('celul',)",86.363636
1,1,"('equip',)",70.000000
2,2,"('celul',)",93.939394
3,3,"('sac',)",62.500000
4,4,"('telefon',)",52.631579
5,5,"('equip',)",100.000000
6,6,"('celul',)",81.250000
7,7,"('equip',)",85.714286
8,8,"('compr',)",100.000000
9,9,"('compr',)",80.769231


In [89]:
#en qué porcentaje de utterances existe el bigrama
tabla_bigramas=tabla_bigramas_clusters.groupby(["cluster"]).agg({'porcentajeBigramaRespectoCluster':'max',"bigramasClusters":"first"})[["bigramasClusters","porcentajeBigramaRespectoCluster"]].reset_index()

In [90]:
tabla_bigramas

,cluster,bigramasClusters,porcentajeBigramaRespectoCluster
0,0,"('rompi', 'celul')",68.181818
1,1,"('sac', 'equip')",20.000000
2,2,"('rompi', 'celul')",90.909091
3,3,"('lleg', 'mensaj')",25.000000
4,4,"('bloqu', 'telefon')",15.789474
5,5,"('lleg', 'equip')",30.000000
6,6,"('perd', 'celul')",25.000000
7,7,"('compr', 'equip')",28.571429
8,8,"('equip', 'compr')",57.142857
9,9,"('compr', 'celul')",42.307692


In [91]:
if "index" in utterances_.columns:
    utterances_.drop(columns=["index"],inplace=True)
if "Unnamed: 0" in utterances_.columns:
    utterances_.drop(columns=["Unnamed: 0"],inplace=True)
if 'level_0' in utterances_.columns:
    utterances_.drop(columns=['level_0'],inplace=True)

In [92]:
if armarSubClusters:
    columnas_nuevas = ['cluster', 'subcluster', 'unigramaCluster',
           'repeticionUnigramasClusters', 'porcentajeUnigramaRespectoCluster',
           'bigramasClusters','repeticionBigramasClusters', 'porcentajeBigramaRespectoCluster']
else:
    columnas_nuevas = ['cluster', 'unigramaCluster',
           'repeticionUnigramasClusters', 'porcentajeUnigramaRespectoCluster',
           'bigramasClusters','repeticionBigramasClusters', 'porcentajeBigramaRespectoCluster']
columnas_final = columnas_originales_del_csv.tolist()
columnas_final.extend(columnas_nuevas) 

exportar=utterances_[columnas_final]

In [93]:
utterances_.cluster.value_counts()

2     33
9     26
14    26
19    23
0     22
4     19
6     16
17    15
8     14
16    13
5     10
11    10
1     10
10     9
3      8
7      7
13     7
18     7
20     6
21     5
12     5
15     5
22     3
23     3
Name: cluster, dtype: int64

### Para exportar excel formateado

In [94]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context
# project = Project.access()


Db2Certificate_metadata = project.get_connection(name="CCC_Db2")
#print (Db2Certificate_metadata)

MongoCertificate_metadata = project.get_connection(name="CCC - MongoDB Produccion")
#print (MongoCertificate_metadata)
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.

################           Obtiene datos de la conexión CCC  ETL (ICOS) #######################
ICOSCertificate_metadata = project.get_connection(name="CCC  ETL")

import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values

COS_ENDPOINT  = ICOSCertificate_metadata['url']
COS_API_KEY_ID  = ICOSCertificate_metadata['api_key']
COS_INSTANCE_CRN  = ICOSCertificate_metadata['resource_instance_id']
BUCKET='ccc-stage'

# Create resource
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [95]:
import io
#funcion para poder exportar un excel
def generarExcel(listaDfs,listaPestaniasExcel,nombreArchivo):
    output = io.BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    
    for dff, pest in zip(listaDfs, listaPestaniasExcel):
        dff.to_excel(writer,sheet_name=pest,index=False)
        
    for i,x in zip(writer.sheets.keys(),listaDfs ):
        # Get the xlsxwriter workbook and worksheet objects.
        workbook = writer.book
        worksheet = writer.sheets[i]

        # Get the dimensions of the dataframe.
        (max_row, max_col) = x.shape

        # Create a list of column headers, to use in add_table().
        column_settings = [{'header': column} for column in x.columns]

        # Add the Excel table structure. Pandas will add the data.
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet.set_column(0, max_col - 1, 12)
        
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

    xlsx_data = output.getvalue()
    
    projectCCoC.save_data(data=xlsx_data,file_name=f"{nombreArchivo}.xlsx",overwrite=True)

    #nombreArchivo= 'Informes-Data-VC/' +  nombreArchivo 
    #cos.Bucket('-ccc-historic').put_object(Key=f"{nombreArchivo}.xlsx", Body=xlsx_data, ContentType='application/excel')


In [96]:
if armarSubClusters:
    generarExcel([exportar,textos_representativos,tabla_unigramas,tabla_bigramas],["utterances","textos_representativos","tabla_unigramas","tabla_bigramas"],nombreArchivoExportar)
else:
    generarExcel([exportar,tabla_unigramas,tabla_bigramas],["utterances","tabla_unigramas","tabla_bigramas"],nombreArchivoExportar)

In [97]:
print ('Si no hubo error, buscar el archivo exportado como: "'+ nombreArchivoExportar + '.xlsx"')

Si no hubo error, buscar el archivo exportado como: "cruce_intenciones-bigramas.xlsx"
